In [ ]:
from utils.binaries import *
from utils.plotting import *
import pickle

from utils.Auger.SD.Triggers import filter_and_downsample, wcd_t1_trigger

In [ ]:
def read_file(path_to_file):

    trace_has_ssd = lambda df : len(df.filter(pl.col('id') == 5)) == 1
    trace_has_all_wcd = lambda df : len(df.filter(
        (pl.col('pmt') == 1) |
        (pl.col('pmt') == 2) |
        (pl.col('pmt') == 3)
        )) == 3

    data = pl.read_csv(path_to_file,
                   has_header=False,
                   skip_rows=1,
                   new_columns=['id', 'pmt', 'baseline'],
                   )
   
    data = data.with_columns(
        zero_trace=pl.concat_list(pl.exclude('id', 'pmt', 'baseline') - pl.col('baseline'))
    )
    data = data.select('id', 'pmt', 'baseline', 'zero_trace')


    for _id, pmt, baseline, trace in data.iter_rows():
        if sum([_bin is None for _bin in trace]):
            data = data.filter(
                pl.col('id') != _id
            )

    return data.group_by(['id'])

def clip_zero_signal(trace):

    pmt1, pmt2, pmt3 = trace
    assert (trace_length := len(pmt1)) == len(pmt2) == len(pmt3), "Unrealistic time trace"

    for start in range(trace_length):
        if pmt1[start] != pmt2[start] != pmt3[start] != 0: break
    for stop in range(-1, -trace_length, -1):
        if pmt1[stop] != pmt2[stop] != pmt3[stop] != 0: break

    return np.array(trace)[:, max(start-10, 0):min(stop+10, trace_length)]

def plot_test_trace(trace, ax):
    pmt1, pmt2, pmt3 = trace
    ax.plot(range(len(pmt1)), pmt1, label='pmt1')
    ax.plot(range(len(pmt1)), pmt2, label='pmt2')
    ax.plot(range(len(pmt1)), pmt3, label='pmt3')

In [ ]:
traces = read_file('/cr/tempdata01/filip/LESimulations/dump_trace-uub-hw_all-full_0-fast_1.dat')

# from /cr/data01/filip/offline/Framework/SDetector/SdSimCalibrationConstants.xml.in
vem_peak = np.array([215.781, 215.781, 215.781])
mip_peak = 51.8

get_wcd_traces = lambda df: df.filter(
    (pl.col('pmt') == 1) |
    (pl.col('pmt') == 2) |
    (pl.col('pmt') == 3)
    ).to_numpy()[:, 3:][:,0]

trace_has_all_wcd = lambda df : len(get_wcd_traces(df)) == 3

t1_peaks, all_peaks = [], []
for _id, trace in traces:
    if len(trace.filter(pl.col('pmt') == 5)) == 0: continue             # no signal in SSD

    if not trace_has_all_wcd(trace): is_t1 = False                      # not all WCD PMTs received signal -> no T1
    else:
        wcd_traces = get_wcd_traces(trace)
        filtered_downsampled = filter_and_downsample(*wcd_traces)
        fds_clipped_trace = clip_zero_signal(filtered_downsampled)
        vem_trace = fds_clipped_trace / vem_peak[:, np.newaxis]

        t1_latch_bin = wcd_t1_trigger(vem_trace, latch_bin=True)

    ssd_trace = trace.filter(pl.col('pmt') == 5)['zero_trace'].to_numpy()[0] / mip_peak

    if t1_latch_bin != -1:
        t1_latch_bin *= 3       # to convert from UUB to UB binning
        ssd_max = ssd_trace[t1_latch_bin - 19 : t1_latch_bin + 40].max()
        t1_peaks.append(ssd_max)
    else:
        ssd_max = ssd_trace.max()

    all_peaks.append(ssd_max)

with open('/cr/tempdata01/filip/LESimulations/all_peaks.pkl', 'wb') as f:
    pickle.dump(all_peaks, f)
with open('/cr/tempdata01/filip/LESimulations/t1_peaks.pkl', 'wb') as f:
    pickle.dump(t1_peaks, f)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)

plot_test_trace(wcd_traces, ax1)
plot_test_trace(vem_trace, ax2)